In [17]:
!pip install -r requirements.txt

  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.4
    Uninstalling pandas-1.3.4:
      Successfully uninstalled pandas-1.3.4


You should consider upgrading via the 'C:\Users\Admin\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [18]:
import pandas as pd

In [27]:
'''
HateSpeechDataset Features:
count: number of CrowdFlower users who coded each tweet (min is 3, sometimes more users coded a tweet when judgments were determined to be unreliable by CF)
hate_speech: number of CF users who judged the tweet to be hate speech
offensive_language: number of CF users who judged the tweet to be offensive
neither: number of CF users who judged the tweet to be neither offensive nor non-offensive
class: class label for majority of CF users. 0 - hate speech 1 - offensive language 2 - neither
tweet: text tweet
'''
HateSpeechDataset = pd.read_csv('datasets/Hate_Speech_and_Offensive_Language_Dataset.csv')


In [28]:
display(HateSpeechDataset.sample(10))
display(HateSpeechDataset.info())
display(HateSpeechDataset.describe())

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
15438,15801,3,0,3,0,1,RT @HardNuckLife: Kamehamehaaaaaaa bitch.
23602,24095,3,0,3,0,1,balling like I'm stunna bitch
15314,15674,3,0,3,0,1,RT @GOLDBONEQUEEN: Stevie J turned a hoe &#128...
7836,8056,3,0,3,0,1,At 6'0? Good luck Nh nh RT @UnnagroundKing: I'...
21970,22437,7,0,7,0,1,This bitch is nasty
3313,3399,3,0,3,0,1,@GrizzboAdams @wyattnuckels this Friday at the...
12922,13241,3,0,3,0,1,Ms St aint no hoe. They fuckin Auburn up
20948,21399,3,1,2,0,1,So fucking thirsty for a championship that the...
1176,1202,3,1,2,0,1,&#8220;@CheyaGurr: @A_Mitchellll13 ur seriousl...
24086,24587,6,0,6,0,1,ion care if I'm in the wrong bitch don't blow ...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


None

,Unnamed: 0,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,12681.192027,3.243473,0.280515,2.413711,0.549247,1.110277
std,7299.553863,0.883060,0.631851,1.399459,1.113299,0.462089
min,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000
25%,6372.500000,3.000000,0.000000,2.000000,0.000000,1.000000
50%,12703.000000,3.000000,0.000000,3.000000,0.000000,1.000000
75%,18995.500000,3.000000,0.000000,3.000000,0.000000,1.000000
max,25296.000000,9.000000,7.000000,9.000000,9.000000,2.000000
